# Solution: The Forgotten Order

This notebook shows the correct solution to the memory overflow drill.

---

In [ ]:
import re
from dataclasses import dataclass
from datetime import datetime
from typing import List, Dict, Optional

In [ ]:
@dataclass
class Message:
    role: str
    content: str
    tokens: int = 0
    
    def __post_init__(self):
        self.tokens = len(self.content) // 4

## The Problem: Broken Memory

The original `BrokenConversationMemory` simply removes oldest messages without checking what critical information they contain.

In [ ]:
class BrokenConversationMemory:
    """Memory that loses important info when context overflows."""
    
    def __init__(self, max_tokens: int = 100):
        self.messages: List[Message] = []
        self.max_tokens = max_tokens
    
    def add(self, role: str, content: str):
        msg = Message(role=role, content=content)
        self.messages.append(msg)
        self._enforce_limit()
    
    def _enforce_limit(self):
        """BUG: Just removes oldest messages, no matter what they contain!"""
        total = sum(m.tokens for m in self.messages)
        while total > self.max_tokens and len(self.messages) > 1:
            removed = self.messages.pop(0)
            total -= removed.tokens
    
    def get_context(self) -> str:
        return "\n".join([f"{m.role}: {m.content}" for m in self.messages])
    
    def get_order_id(self) -> Optional[str]:
        context = self.get_context()
        match = re.search(r'ORD-\d+', context)
        return match.group() if match else None

## The Solution: Entity-Aware Memory

The fix extracts and stores critical entities separately, then prepends them to the context.

In [ ]:
class EntityAwareMemory:
    """Memory that preserves critical entities."""
    
    def __init__(self, max_tokens: int = 100):
        self.messages: List[Message] = []
        self.max_tokens = max_tokens
        self.entities = {}  # Track extracted entities
    
    def add(self, role: str, content: str):
        msg = Message(role=role, content=content)
        self.messages.append(msg)
        self._extract_entities(content)
        self._enforce_limit()
    
    def _extract_entities(self, text: str):
        """Extract and save important entities."""
        # Extract order IDs
        orders = re.findall(r'ORD-\d+', text, re.IGNORECASE)
        for order in orders:
            self.entities['order_id'] = order.upper()
        
        # Extract emails
        emails = re.findall(r'[\w.-]+@[\w.-]+\.\w+', text)
        for email in emails:
            self.entities['email'] = email
        
        # Extract amounts
        amounts = re.findall(r'\$[\d,]+\.?\d*', text)
        if amounts:
            self.entities['amount'] = amounts[-1]
    
    def _enforce_limit(self):
        """Remove old messages but preserve entities."""
        total = sum(m.tokens for m in self.messages)
        while total > self.max_tokens and len(self.messages) > 1:
            removed = self.messages.pop(0)
            total -= removed.tokens
    
    def get_context(self) -> str:
        """Get context with preserved entities."""
        # Add entity summary at the start
        entity_summary = ""
        if self.entities:
            entity_summary = "Key facts: " + ", ".join([f"{k}={v}" for k,v in self.entities.items()]) + "\n\n"
        
        messages = "\n".join([f"{m.role}: {m.content}" for m in self.messages])
        return entity_summary + messages
    
    def get_order_id(self) -> Optional[str]:
        """Get order ID from entity store."""
        return self.entities.get('order_id')

print("✓ EntityAwareMemory defined")

## Comparing the Results

In [ ]:
# Test conversation
conversation = [
    ("user", "Hi, I need help with order ORD-12345"),
    ("assistant", "I'd be happy to help with order ORD-12345! What's the issue?"),
    ("user", "The item arrived damaged"),
    ("assistant", "I'm sorry to hear that. I can arrange a replacement."),
    ("user", "How long will that take?"),
    ("assistant", "Replacements typically arrive in 3-5 business days."),
    ("user", "Can you also check if I have any coupons?"),
    ("assistant", "Let me check your account for available coupons."),
    ("user", "Great, and when will my replacement ship?"),
]

# Test broken memory
broken = BrokenConversationMemory(max_tokens=100)
for role, content in conversation:
    broken.add(role, content)

print("=== Broken Memory ===")
print(f"Order ID: {broken.get_order_id()}")
print(f"Messages remaining: {len(broken.messages)}")

# Test fixed memory
fixed = EntityAwareMemory(max_tokens=100)
for role, content in conversation:
    fixed.add(role, content)

print("\n=== Fixed Memory ===")
print(f"Order ID: {fixed.get_order_id()}")
print(f"Messages remaining: {len(fixed.messages)}")
print(f"Entities preserved: {fixed.entities}")

In [ ]:
print("\n=== Context with Entities ===")
print(fixed.get_context())

In [ ]:
# Verification
assert fixed.get_order_id() == "ORD-12345", "Should preserve order ID"
assert "ORD-12345" in fixed.get_context(), "Context should include order ID"

print("\n✓ All checks passed!")
print("✓ Order ID preserved despite context overflow")

## Key Insight

The fix works by:

1. **Extracting entities as they appear** - Order IDs, emails, amounts are stored separately
2. **Storing them outside the sliding window** - Entities persist even when their source messages are removed
3. **Prepending to context** - Every response includes "Key facts: order_id=ORD-12345"

This ensures the model always has access to critical information, even in long conversations.